In [1]:
from sqlanalyzer import column_parser, unbundle
from sqlanalyzer.unbundle import *
import sqlparse
import re
import json
import pandas as pd
import time


def extract_subquery_fields(query, db_fields):
    formatter = column_parser.Parser(query)
    formatted = formatter.format_query(query)
    fields = formatter.match_queried_fields(formatted, db_fields)
    return fields


def compile_queried_cols(query_dict, df):
    all_cols = []
    for _,v in query_dict.items():
        if isinstance(v, dict):
            for _,v1 in v.items():
                all_cols.extend(extract_subquery_fields(v1, df))
        else:
            all_cols.extend(extract_subquery_fields(v, df))
    return all_cols

In [2]:
query = open('active_devs.sql').read()

formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')

unbundled = unbundle.Unbundle(query)


In [3]:
if is_cte(formatted_query):
    cte_dict = formatter.parse_cte(formatted_query)
    final_dict = {}
    for alias, query in cte_dict.items():
        formatter = column_parser.Parser(query)
        formatted_query = formatter.format_query(query)
        try:
            final_dict[alias] = unbundled.extract_query_dict(formatted_query)
        except:
            final_dict[alias] = formatted_query

else:
    final_dict = unbundled.extract_query_dict(query)

        

In [5]:
for k,v in cte_dict.items():
    print(k, '\n', v, '\n\n')

analytics_service_endpoint_mapping 
 SELECT b.*
   FROM
     (SELECT MAX(dt) AS dt
      FROM analytics.service_endpoint_mapping) a
   INNER JOIN analytics.service_endpoint_mapping b ON a.dt = b.dt),
 


web_dev 
 SELECT s.dt,
          COALESCE(em.platform, 'unknown') AS platform,
          COALESCE(service_org, 'other') AS service,
          account
   FROM sku.daily_by_account s
   INNER JOIN mapbox_customer_data.accounts a ON s.account = a.id
   AND a.dt = '{run_date}'
   LEFT JOIN analytics_service_endpoint_mapping em ON s.sku_id = em.sku_id
   AND em.in_sku IS NOT NULL
   AND em.parent_sku IS NULL
   WHERE s.dt BETWEEN DATE_SUB('{run_date}', 29) AND '{run_date}'
   GROUP BY 1,
            2,
            3,
            4),
 


mobile_dev 
 SELECT dt,
          'mobile' AS platform,
          CASE
              WHEN LOWER(useragent) RLIKE '(mapboxeventsnavigation|navigation-)' THEN 'navigation'
              WHEN LOWER(useragent) RLIKE '(mapboxeventsunity|mapbox-scenekit)' THEN 'un

In [6]:
cte_dict.keys()

dict_keys(['analytics_service_endpoint_mapping', 'web_dev', 'mobile_dev', 'studio_dev', 'web_mobile_studio', 'mau_cube', 'wau_cube', 'dau_cube', 'main'])

In [39]:
cte_dict['analytics_service_endpoint_mapping']

'SELECT b.*\n   FROM\n     (SELECT MAX(dt) AS dt\n      FROM analytics.service_endpoint_mapping) a\n   INNER JOIN analytics.service_endpoint_mapping b ON a.dt = b.dt),\n'

In [40]:
# for alias, query in cte_dict.items():
final_dict = {}
query = cte_dict['analytics_service_endpoint_mapping']
alias = 'analytics_service_endpoint_mapping'
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
final_dict[alias] = unbundled.extract_query_dict(formatted_query)

    

In [41]:
# extract_query_dict
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list_0 = formatted_query.split('\n')
query_dict = {}

sub_query = unbundled.delevel(query_list_0)
query_dict = sub_query
query_dict

{'b': 'analytics.service_endpoint_mapping) a INNER JOIN analytics.service_endpoint_mapping',
 'main': 'SELECT b.* b ON a.dt = b.dt),'}

In [42]:
final_dict

{'analytics_service_endpoint_mapping': {'b': 'analytics.service_endpoint_mapping) a INNER JOIN analytics.service_endpoint_mapping',
  'main': 'SELECT b.* b ON a.dt = b.dt),'}}

In [43]:
formatted_query

'SELECT b.*\nFROM\n  (SELECT MAX(dt) AS dt\n   FROM analytics.service_endpoint_mapping) a\nINNER JOIN analytics.service_endpoint_mapping b ON a.dt = b.dt),'

In [ ]:
# query_list_1 = formatted_query.split('\n')
# query_dict = {}

# sub_query = unbundled.delevel(query_list_0)
# query_dict = sub_query


In [ ]:
# query_dict

In [ ]:
# query_1 = cte_dict['wau_cube']
# formatter = column_parser.Parser(query_1)
# formatted_query_1 = formatter.format_query(query_1)
# query_list_1 = formatted_query_1.split('\n')


In [ ]:
# query_2 = cte_dict['web_dev']
# formatter = column_parser.Parser(query_2)
# formatted_query_2 = formatter.format_query(query_2)
# query_list_2 = formatted_query_2.split('\n')


In [ ]:
# for i,l in enumerate(query_list_1): print(i, l)

In [ ]:
# for i,l in enumerate(query_list_2): print(i, l)

In [44]:
query = cte_dict['analytics_service_endpoint_mapping']
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')
query_list

['SELECT b.*',
 'FROM',
 '  (SELECT MAX(dt) AS dt',
 '   FROM analytics.service_endpoint_mapping) a',
 'INNER JOIN analytics.service_endpoint_mapping b ON a.dt = b.dt),']

In [45]:
for i, l in enumerate(query_list): print(i, l)

0 SELECT b.*
1 FROM
2   (SELECT MAX(dt) AS dt
3    FROM analytics.service_endpoint_mapping) a
4 INNER JOIN analytics.service_endpoint_mapping b ON a.dt = b.dt),


In [13]:
pos_delete, pos_where = [len(query_list)-1], len(query_list)
pos_join = []
for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)
    elif line.startswith('FROM') and len(line.split(' ')) > 1:
        pos_join.append(i)
        
    elif line.startswith('FROM') and len(line.split(' ')) == 1:

        for i2,line2 in enumerate(query_list[i+1:]):
            if line2.startswith(' '): pos_join.append(i2+i+1)
            else: break

            
    elif line.startswith('WHERE'):
        pos_where = i
    elif line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN') or line.startswith('RIGHT JOIN'):
        pos_join.append(i)

if min(pos_delete) == len(query_list)-1:
    pos_join.append(min(pos_delete))
else:
    pass

In [14]:
pos_join

[2, 3, 4, 4]

In [15]:
pos_where

5

In [16]:
# delevel

sub_query = {}
pos_join, pos_where = unbundled._get_joins_pos(query_list)
pos_join, pos_where


([2, 3, 4], 5)

In [17]:
alias_pos = unbundled._get_alias_pos(query_list, pos_join, pos_where)
alias_pos

[1, 4]

## _get_alias_pos

In [18]:
pos_join = list(set(pos_join))
pos_join_list = iter(pos_join)
next(pos_join_list)
alias_pos = []


In [19]:
if query_list[pos_join[0]].startswith('FROM'):
    alias_pos.append(pos_join[0])


In [20]:
pos_join

[2, 3, 4]

In [21]:
for i in range(len(pos_join)-1):
    if i < len(pos_join)-2 and pos_join[i] < pos_where:
        end_pos = next(pos_join_list)-1
        alias_pos.append(end_pos-1)

    elif pos_join[-1] >= pos_where:
        end_pos = next(pos_join_list)-1
        alias_pos.append(pos_where - 1)

    else:
        end_pos = pos_join[-1]
        alias_pos.append(pos_where-1)


In [22]:
alias_pos = sorted(list(set(alias_pos)))

In [23]:
alias_pos

[1, 4]

In [24]:
sub_query_pos = list(zip(pos_join, alias_pos))
sub_query_pos

[(2, 1), (3, 4)]

In [25]:
sub_query, keep = unbundled._parse_sub_query(query_list, sub_query_pos)
sub_query, keep

({'b': 'analytics.service_endpoint_mapping) a INNER JOIN analytics.service_endpoint_mapping'},
 ['b ON a.dt = b.dt),'])

In [26]:
main_query_pos = unbundled.main_query(query_list, sub_query_pos)
main_query_pos

[0]

In [27]:
if main_query_pos != []:
    sub_query['main'] = '\n'.join([query_list[p] for p in main_query_pos])
sub_query

{'b': 'analytics.service_endpoint_mapping) a INNER JOIN analytics.service_endpoint_mapping',
 'main': 'SELECT b.*'}

In [28]:
sub_query['main'] = sub_query['main'] + ' ' + '\n'.join(keep)
sub_query

{'b': 'analytics.service_endpoint_mapping) a INNER JOIN analytics.service_endpoint_mapping',
 'main': 'SELECT b.* b ON a.dt = b.dt),'}